# MC886 Assigment 1

## Objective:
Use linear regression model to predict the prices of diamonds given their atributes. The data set might be found on:
https://www.kaggle.com/shivam2503/diamonds.  


In [26]:
from __future__ import print_function
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [11]:
data_dir = '../data/diamonds.csv'

In [12]:
with open(data_dir, 'rb') as csvfile:
    dataset = pd.read_csv(data_dir)
    
print(type(dataset))

<class 'pandas.core.frame.DataFrame'>


In [74]:
#carat	cut	color	clarity	depth	table	price	x	y	z
print(dataset['cut'])

0            Ideal
1          Premium
2             Good
3          Premium
4             Good
5        Very Good
6        Very Good
7        Very Good
8             Fair
9        Very Good
10            Good
11           Ideal
12         Premium
13           Ideal
14         Premium
15         Premium
16           Ideal
17            Good
18            Good
19       Very Good
20            Good
21       Very Good
22       Very Good
23       Very Good
24       Very Good
25       Very Good
26         Premium
27       Very Good
28       Very Good
29       Very Good
           ...    
53910      Premium
53911      Premium
53912      Premium
53913         Good
53914         Good
53915        Ideal
53916         Good
53917    Very Good
53918      Premium
53919        Ideal
53920    Very Good
53921    Very Good
53922    Very Good
53923        Ideal
53924        Ideal
53925        Ideal
53926        Ideal
53927         Good
53928      Premium
53929        Ideal
53930      Premium
53931      P

## Preprocessing the data

### 1. Split data in training, validation and test
A wise person said: "friends don’t let friends use testing data for training".  Set bellow the sizes of each set.

**Note:**
* **sklearn.model_selection.train_test_split(*arrays, **options)**  
    Split arrays or matrices into random train and test subsets.  
    See documentation:http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

### 2. Put the dataset into Numpy volume
Categorical atributes are changed to integers.

In [81]:
#1. SPLIT DATASET
data_size = 53940        #change values here
train_size = 30000
validation_size = 10000
test_size = 13940

nx = 9                   #number of features of the input
ny = 1                   #number of fetures of the output

train,test_validation = train_test_split(dataset, test_size = validation_size + test_size)
validation, test = train_test_split(test_validation, test_size = test_size)

print ("data_size: ", data_size)
print ("train_size: ", train.shape[0])
print ("validation_size: ", validation.shape[0])
print ("test_size: ", test.shape[0])


#2. SAVE THE PANDA'S DATAFRAME ON NUMPY ARRAYS
categorical_labels = ['carat', 'cut', 'color', 'clarity', 'depth', 'table', 'price', 'x', 'y', 'z']

train_volume = np.empty(train.shape)
validation_volume = np.empty(validation.shape)
test_volume = np.empty(test.shape)

# For each feature, copy or transforms and copy (in categorical case) to the correspondent volume
for i in range(len(categorical_labels)):
    if( i >= 1 and i <= 3): #if the feature is categorical
        train_volume [:,i] = label_encoder.fit_transform(train[categorical_labels[i]])
        validation_volume[:, i] = label_encoder.fit_transform(validation[categorical_labels[i]])
        test_volume[:, i] = label_encoder.fit_transform(test[categorical_labels[i]])
    else:
        train_volume[:,i] = train[categorical_labels[i]]
        validation_volume[:,i] = validation[categorical_labels[i]]
        test_volume[:,i] = test[categorical_labels[i]]

# Separates the input from the label organize data in (number_features, number_of_examples)
x_train = (train_volume[:, 0:9]).T
y_train = (train_volume[:, 9].reshape(train_size,1)).T
x_validation = validation_volume [:, 0:9].T
y_validation = validation_volume [:,9].reshape(ny, validation_size)
x_test = test_volume [:,0:9].T
y_test = test_volume [:,9].reshape(test_size,1).T

# Sanity check
assert(x_train.shape == (nx, train_size))
assert(y_train.shape == (ny, train_size))
assert(x_validation.shape == (nx, validation_size))
assert(y_validation.shape == (ny, validation_size))
assert(x_test.shape == (nx, test_size))
assert(y_test.shape == (ny, test_size))

data_size:  53940
train_size:  30000
validation_size:  10000
test_size:  13940
